# Overview

In this notebook, we illustrate some commonly used augmentations for images in various libraries, and perform inference with both TensorFlow and PyTorch pipelines.

A key augmentation that we will like to illustrate is the needle augmentation, or rather an augmentation that performs an overlay of small artifacts onto the images with a certain probability.

A variant of this approach first appeared in the recent Melanoma Detection competition, created by Roman. I've since adapted it in prior and other competitions such as the Global Wheat Detection competition.

# Summary of Findings (Using Needle Augmentation)

- PyTorch:
    - when applied to @yasufuminakama pipeline with ResNeXt50, boosted LB from 0.949 to 0.953 with 5-folds CV, using same hyperparameters
    - when using 10-folds with same hyperparameters, LB became 0.954
- TensorFlow-Keras:
    - when applied to @xhlulu pipeline with EfficientNetB7, LB got boosted from 0.957 to 0.959 using same hyperparameters

# Imports

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
import tensorflow as tf
import albumentations as A

In [ ]:
DIR = "../input/ranzcr-clip-catheter-line-classification/"

In [ ]:
df_annotations = pd.read_csv(os.path.join(DIR, "train_annotations.csv"))

# Read Sample Image without Augmentations

In [ ]:
row = df_annotations.iloc[8]
image_path = os.path.join(DIR, "train", row["StudyInstanceUID"] + ".jpg")
chosen_image = cv2.imread(image_path)

# Albumentations Augmentations

- Albumentations part adapted from my good friend Hongnan's notebbok in the Global Wheat Detection competition (https://www.kaggle.com/reighns/augmentations-data-cleaning-and-bounding-boxes#Bounding-Boxes-with-Albumentations)
- Added more augmentations which may be useful
- Added TensorFlow and Torchvision versions of the augmentations

In [ ]:
albumentation_list = [A.RandomSunFlare(p=1), 
                      A.RandomFog(p=1), 
                      A.RandomBrightness(p=1),
                      A.RandomCrop(p=1,height = 512, width = 512), 
                      A.Rotate(p=1, limit=90),
                      A.RGBShift(p=1), 
                      A.RandomSnow(p=1),
                      A.HorizontalFlip(p=1), 
                      A.VerticalFlip(p=1), 
                      A.RandomContrast(limit = 0.5,p = 1),
                      A.HueSaturationValue(p=1,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50),
                      A.Cutout(p=1),
                      A.Transpose(p=1), 
                      A.JpegCompression(p=1),
                      A.CoarseDropout(p=1),
                      A.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=1),
                      A.IAAAffine(scale=1.0, translate_percent=None, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', p=1),
                      A.IAAAffine(rotate=90., p=1),
                      A.IAAAffine(rotate=180., p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in albumentation_list:
    img = aug_type(image = chosen_image)['image']
    img_matrix_list.append(img)

img_matrix_list.insert(0,chosen_image)    

titles_list = ["Original","RandomSunFlare","RandomFog","RandomBrightness",
               "RandomCrop","Rotate", "RGBShift", "RandomSnow","HorizontalFlip", "VerticalFlip", "RandomContrast","HSV",
               "Cutout","Transpose","JpegCompression","CoarseDropout","IAAAdditiveGaussianNoise","IAAAffine","IAAAffineRotate90","IAAAffineRotate180"]

def plot_multiple_img(img_matrix_list, title_list, ncols, nrows=5,  main_title=""):
    fig, myaxes = plt.subplots(figsize=(20, 15), nrows=nrows, ncols=ncols, squeeze=False)
    fig.suptitle(main_title, fontsize = 30)
    fig.subplots_adjust(wspace=0.3)
    fig.subplots_adjust(hspace=0.3)
    for i, (img, title) in enumerate(zip(img_matrix_list, title_list)):
        myaxes[i // ncols][i % ncols].imshow(img)
        myaxes[i // ncols][i % ncols].set_title(title, fontsize=15)
    plt.show()
    
plot_multiple_img(img_matrix_list, titles_list, ncols = 4,main_title="Different Types of Augmentations with Albumentations")

# OpenCV-Based Custom Augmentations

## Xray Needle Augmentation

- Originally, I implemented this for the recent Global Wheat Detection competition at https://www.kaggle.com/khoongweihao/insect-augmentation-with-efficientdet-d6, inspired by Roman's hair augmentation from the recent Melanoma competition
- Motivations for this approach:
    - the intuition was to mimic as close to as possible, actual scenarios in reality that can happen
    - needles were used as an example. Other small artifacts such as intubation or bullets (due to gunshot wounds) overlayed with the images should suffice. Main idea is to mimic what we can observe in reality, in the form of augmentations

In [ ]:
def NeedleAugmentation(image, n_needles=2, dark_needles=False, p=0.5, needle_folder='../input/xray-needle-augmentation'):
    aug_prob = random.random()
    if aug_prob < p:
        height, width, _ = image.shape  # target image width and height
        needle_images = [im for im in os.listdir(needle_folder) if 'png' in im]

        for _ in range(1, n_needles):
            needle = cv2.cvtColor(cv2.imread(os.path.join(needle_folder, random.choice(needle_images))), cv2.COLOR_BGR2RGB)
            needle = cv2.flip(needle, random.choice([-1, 0, 1]))
            needle = cv2.rotate(needle, random.choice([0, 1, 2]))

            h_height, h_width, _ = needle.shape  # needle image width and height
            roi_ho = random.randint(0, abs(image.shape[0] - needle.shape[0]))
            roi_wo = random.randint(0, abs(image.shape[1] - needle.shape[1]))
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(needle, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of needle in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_needles:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                needle_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                needle_fg = cv2.bitwise_and(needle, needle, mask=mask)

            # Put needle in ROI and modify the target image
            dst = cv2.add(img_bg, needle_fg, dtype=cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

    return image

### Original Needles

In [ ]:
chosen_image = cv2.imread(image_path)
aug_image = NeedleAugmentation(chosen_image, n_needles=3, dark_needles=False, p=1.0)
plt.imshow(aug_image)

### Dark Needles (Just black, dark, sinister as it is on an Xray...)

In [ ]:
chosen_image = cv2.imread(image_path)
aug_image = NeedleAugmentation(chosen_image, n_needles=3, dark_needles=True, p=1.0)
plt.imshow(aug_image)

# PyTorch-Based Augmentations (Torchvision)

In [ ]:
torch_trans_list = [transforms.CenterCrop((178, 178)),
                    transforms.Resize(128),
                    transforms.RandomRotation(45),
                    transforms.RandomAffine(35),
                    transforms.RandomCrop(128),
                    transforms.RandomHorizontalFlip(p=1),
                    transforms.RandomPerspective(p=1),
                    transforms.RandomVerticalFlip(p=1)]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_type in torch_trans_list:
    # convert to tensor
    chosen_image = cv2.imread(image_path)
    chosen_tensor = transforms.Compose([transforms.ToTensor()])(chosen_image)
    chosen_tensor = transforms.Compose([aug_type])(chosen_tensor)
    trans_img = transforms.ToPILImage()(chosen_tensor)
    img_matrix_list.append(trans_img)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","CenterCrop","Resize","RandomRotation","RandomAffine","RandomCrop","RandomHorizontalFlip","RandomPerspective",
               "RandomVerticalFlip"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=3, main_title="Different Types of Augmentations with Albumentations")

# TensorFlow-Based Augmentations

In [ ]:
chosen_image = cv2.imread(image_path)

tf_trans_list = [
    tf.image.rot90(chosen_image, k=1), # 90 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=2), # 180 degrees counter-clockwise
    tf.image.rot90(chosen_image, k=3), # 270 degrees counter-clockwise
    tf.image.random_brightness(chosen_image, 0.5), 
    tf.image.random_contrast(chosen_image, 0.2, 0.5), 
    tf.image.random_flip_left_right(chosen_image, seed=42),
    tf.image.random_flip_up_down(chosen_image, seed=42),
    tf.image.random_hue(chosen_image, 0.5),
    tf.image.random_jpeg_quality(chosen_image, 35, 50), 
    tf.image.random_saturation(chosen_image, 5, 10), 
    tf.image.transpose(chosen_image),
]

In [ ]:
img_matrix_list = []
bboxes_list = []
for aug_image in tf_trans_list:
    img_matrix_list.append(aug_image)

img_matrix_list.insert(0, chosen_image)    

titles_list = ["Original","Rotate90","Rotate180","Rotate270","RandomBrightness","RandomContrast","RandomLeftRightFlip","RandomUpDownFlip",
               "RandomHue","RandomJPEGQuality","RandomSaturation","Transpose"]

plot_multiple_img(img_matrix_list, titles_list, ncols = 3, nrows=4, main_title="Different Types of Augmentations with Albumentations")

# Model Pipelines with Needle Augmentations

In [ ]:
# select which to inference from
INFER_MODE = 'TF' # or 'TORCH'

## PyTorch

- An example inference using needle augmentation built upon @yasufuminakama inference pipeline at https://www.kaggle.com/yasufuminakama/ranzcr-resnext50-32x4d-starter-inference
- 10 folds were used, LB 0.954 (original baseline by yasufuminakama had LB 0.949 with 5-folds, no needle aug)
- can't seem to perform inference here as it is, having out of memory issues

In [ ]:
!pip install ../input/timm-package/timm-0.1.26-py3-none-any.whl

In [ ]:
if INFER_MODE == 'TORCH':
    import os
    import sys
    sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

    MODEL_DIR = '../input/ranzcr-pytorch-weights/'
    OUTPUT_DIR = './'
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'

In [ ]:
class CFG:
    debug=False
    num_workers=4
    model_name='resnext50_32x4d'
    size=600
    batch_size=64
    seed=42
    target_size=11
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']
    n_fold=10
    trn_fold=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
if INFER_MODE == 'TORCH':
    import math
    import time
    import random
    import shutil
    from pathlib import Path
    from contextlib import contextmanager
    from collections import defaultdict, Counter

    import scipy as sp
    import numpy as np
    import pandas as pd

    from sklearn import preprocessing
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

    from tqdm.auto import tqdm
    from functools import partial

    import cv2
    from PIL import Image

    from matplotlib import pyplot as plt

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.optim import Adam, SGD
    import torchvision.models as models
    from torch.nn.parameter import Parameter
    from torch.utils.data import DataLoader, Dataset
    from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

    from albumentations import (
        Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
        RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
        IAAAdditiveGaussianNoise, Transpose
        )
    from albumentations.pytorch import ToTensorV2
    from albumentations import ImageOnlyTransform

    import timm

    from torch.cuda.amp import autocast, GradScaler

    import warnings 
    warnings.filterwarnings('ignore')

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def get_score(y_true, y_pred):
        scores = []
        for i in range(y_true.shape[1]):
            score = roc_auc_score(y_true[:,i], y_pred[:,i])
            scores.append(score)
        avg_score = np.mean(scores)
        return avg_score, scores


    def get_result(result_df):
        preds = result_df[[f'pred_{c}' for c in CFG.target_cols]].values
        labels = result_df[CFG.target_cols].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {np.round(scores, decimals=4)}')


    @contextmanager
    def timer(name):
        t0 = time.time()
        LOGGER.info(f'[{name}] start')
        yield
        LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


    def init_logger(log_file=OUTPUT_DIR+'inference.log'):
        from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=log_file)
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = init_logger()

    def seed_torch(seed=42):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

    seed_torch(seed=CFG.seed)

    train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
    folds = train.copy()
    Fold = GroupKFold(n_splits=CFG.n_fold)
    groups = folds['PatientID'].values
    for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_cols], groups)):
        folds.loc[val_index, 'fold'] = int(n)
    folds['fold'] = folds['fold'].astype(int)
    display(folds.groupby('fold').size())

    test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

    if CFG.debug:
        test = test.head()

    class TestDataset(Dataset):
        def __init__(self, df, transform=None):
            self.df = df
            self.file_names = df['StudyInstanceUID'].values
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            file_name = self.file_names[idx]
            file_path = f'{TEST_PATH}/{file_name}.jpg'
            image = cv2.imread(file_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            return image

    def get_transforms(*, data):

        if data == 'train':
            return Compose([
                Resize(CFG.size, CFG.size),
                Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                ),
                ToTensorV2(),
            ])

        elif data == 'valid':
            return Compose([
                Resize(CFG.size, CFG.size),
                Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                ),
                ToTensorV2(),
            ])

    class CustomResNext(nn.Module):
        def __init__(self, model_name='resnext50_32x4d', pretrained=False):
            super().__init__()
            self.model = timm.create_model(model_name, pretrained=pretrained)
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, CFG.target_size)

        def forward(self, x):
            x = self.model(x)
            return x

    def inference(model, states, test_loader, device):
        model.to(device)
        tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
        probs = []
        for i, (images) in tk0:
            images = images.to(device)
            avg_preds = []
            for state in states:
                model.load_state_dict(state['model'])
                model.eval()
                with torch.no_grad():
                    y_preds = model(images)
                avg_preds.append(y_preds.sigmoid().to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probs.append(avg_preds)
        probs = np.concatenate(probs)
        return probs

In [ ]:
if INFER_MODE == 'TORCH':
    model = CustomResNext(CFG.model_name, pretrained=False)
    states = [torch.load(MODEL_DIR+f'needle_more_augs_10folds_pretrained_{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
    test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
    test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                             num_workers=CFG.num_workers, pin_memory=True)
    predictions = inference(model, states, test_loader, device)

    test[CFG.target_cols] = predictions
    test[['StudyInstanceUID'] + CFG.target_cols].to_csv(OUTPUT_DIR+'submission.csv', index=False)

## TensorFlow-Keras

- an example inference pipeline with needle augmentation using @xhlulu's notebook from https://www.kaggle.com/xhlulu/ranzcr-efficientnet-submission
- hyperparameters in training were kept constant, LB boosted from 0.957 to 0.959 (maybe just lucky, as its not seeded)

In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [ ]:
if INFER_MODE == 'TF':
    import os
    import efficientnet.tfkeras as efn
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    def auto_select_accelerator():
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("Running on TPU:", tpu.master())
        except ValueError:
            strategy = tf.distribute.get_strategy()
        print(f"Running on {strategy.num_replicas_in_sync} replicas")

        return strategy


    def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
        def decode(path):
            file_bytes = tf.io.read_file(path)
            if ext == 'png':
                img = tf.image.decode_png(file_bytes, channels=3)
            elif ext in ['jpg', 'jpeg']:
                img = tf.image.decode_jpeg(file_bytes, channels=3)
            else:
                raise ValueError("Image extension not supported")

            img = tf.cast(img, tf.float32) / 255.0
            img = tf.image.resize(img, target_size)

            return img

        def decode_with_labels(path, label):
            return decode(path), label

        return decode_with_labels if with_labels else decode


    def build_augmenter(with_labels=True):
        def augment(img):
            # insert needle augmentation here
            img = NeedleAugmentation(img, n_needles=2, dark_needles=False, p=0.5)
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            return img

        def augment_with_labels(img, label):
            return augment(img), label

        return augment_with_labels if with_labels else augment


    def build_dataset(paths, labels=None, bsize=32, cache=True,
                      decode_fn=None, augment_fn=None,
                      augment=True, repeat=True, shuffle=1024, 
                      cache_dir=""):
        if cache_dir != "" and cache is True:
            os.makedirs(cache_dir, exist_ok=True)

        if decode_fn is None:
            decode_fn = build_decoder(labels is not None)

        if augment_fn is None:
            augment_fn = build_augmenter(labels is not None)

        AUTO = tf.data.experimental.AUTOTUNE
        slices = paths if labels is None else (paths, labels)

        dset = tf.data.Dataset.from_tensor_slices(slices)
        dset = dset.map(decode_fn, num_parallel_calls=AUTO)
        dset = dset.cache(cache_dir) if cache else dset
        dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
        dset = dset.repeat() if repeat else dset
        dset = dset.shuffle(shuffle) if shuffle else dset
        dset = dset.batch(bsize).prefetch(AUTO)

        return dset
    
    COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
    strategy = auto_select_accelerator()
    BATCH_SIZE = strategy.num_replicas_in_sync * 16
    
    IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600)

    load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
    sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
    test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

    # Get the multi-labels
    label_cols = sub_df.columns[1:]

    test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[7], IMSIZE[7]))
    dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )
    
    with strategy.scope():
        model = tf.keras.models.load_model(
            '../input/ranzcr-tpu-weights/efficientnetb7-tpu-needle-aug-resume-train.h5'
        )

    model.summary()
    
    sub_df[label_cols] = model.predict(dtest, verbose=1)
    sub_df.to_csv('submission.csv', index=False)

# Useful References

- https://www.tensorflow.org/api_docs/python/tf/image/
- https://albumentations.ai/docs/api_reference/augmentations/transforms/
- https://pytorch.org/docs/stable/torchvision/transforms.html

# Some Remarks

- the needle augmentation implementation still remains buggy with the TensorFlow-Keras pipeline
- Torch pipeline seemed to work perfectly fine, but TF pipeline does raise exceptions once in a while on some seeds
- using insect augmentation (bees instead of needles) in the recent cassava competition showed that it helped stabilize the difference between public and private LB scores

## Do let me know if you have any comments and recommendations going forward! All the best!